In [180]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from utils import *

In [181]:
# superclass of modules
class Module:
    """
    Module is a super class. It could be a single layer, or a multilayer perceptron.
    """
    
    def __init__(self):
        self.train = True
        return
    
    def forward(self, _input):
        """
        h = f(z); z is the input, and h is the output.
        
        Inputs:
        _input: z
        
        Returns:
        output h
        """
        pass
    
    def backward(self, _input, _gradOutput):
        """
        Compute:
        gradient w.r.t. _input
        gradient w.r.t. trainable parameters
        
        Inputs:
        _input: z
        _gradOutput: dL/dh
        
        Returns:
        gradInput: dL/dz
        """
        pass
        
    def parameters(self):
        """
        Return the value of trainable parameters and its corresponding gradient (Used for grandient descent)
        
        Returns:
        params, gradParams
        """
        pass
    
    def training(self):
        """
        Turn the module into training mode.(Only useful for Dropout layer)
        Ignore it if you are not using Dropout.
        """
        self.train = True
        
    def evaluate(self):
        """
        Turn the module into evaluate mode.(Only useful for Dropout layer)
        Ignore it if you are not using Dropout.
        """
        self.train = False

In [182]:
class Sequential(Module):
    """
    Sequential provides a way to plug layers together in a feed-forward manner.
    """
    def __init__(self):
        Module.__init__(self)
        self.layers = [] # layers contain all the layers in order
    
    def add(self, layer):
        self.layers.append(layer) # Add another layer at the end
    
    def size(self):
        return len(self.layers) # How many layers.
    
    def forward(self, _input):
        """
        Feed forward through all the layers, and return the output of the last layer
        """
        # self._inputs saves the input of each layer
        # self._inputs[i] is the input of i-th layer
        self._inputs = [_input]
        
        # YOUR CODE HERE

        for layer in self.layers:
            _input = layer.forward(_input)
            self._inputs.append(_input)
        self._output = self._inputs[-1]

        return self._output
    
    def backward(self, _input, _gradOutput):
        """
        Backpropogate through all the layers using chain rule.
        """
        # self._gradInputs[i] is the gradient of loss w.r.t. the input of i-th layer
        self._gradInputs = [None] * (self.size() + 1)
        self._gradInputs[self.size()] = _gradOutput

        # YOUR CODE HERE

        for i in reversed(range(self.size())):
            _input = self._inputs[i]
            _gradOutput = self.layers[i].backward(_input, _gradOutput)
            self._gradInputs[i] = _gradOutput
        self._gradInput = self._gradInputs[0]
        
        return self._gradInput
    
    def parameters(self):
        """
        Return trainable parameters and its corresponding gradient in a nested list
        """
        params = []
        gradParams = []
        for m in self.layers:
            _p, _g = m.parameters()
            if _p is not None:
                params.append(_p)
                gradParams.append(_g)
        return params, gradParams

    def training(self):
        """
        Turn all the layers into training mode
        """
        Module.training(self)
        for m in self.layers:
            m.training()
    
    def evaluate(self):
        """
        Turn all the layers into evaluate mode
        """
        Module.evaluate(self)
        for m in self.layers:
            m.evaluate()

In [183]:
class FullyConnected(Module):
    """
    Fully connected layer
    """
    def __init__(self, inputSize, outputSize):
        Module.__init__(self)
        # Initalization
        stdv = 1./np.sqrt(inputSize)
        
        self.weight = np.random.uniform(-stdv, stdv, (inputSize, outputSize))
        self.gradWeight = np.ndarray((inputSize, outputSize))
        self.bias = np.random.uniform(-stdv, stdv, outputSize)
        self.gradBias = np.ndarray(outputSize)
        
    def forward(self, _input):
        """
        output = W * input + b
        
        _input:
        N x inputSize matrix
        
        """
        self._output = np.dot(_input, self.weight) + self.bias # YOUR CODE HERE
        return self._output
    
    def backward(self, _input, _gradOutput):
        """
        _input:
        N x inputSize matrix
        _gradOutputSize:
        N x outputSize matrix
        """
        self.gradWeight = np.dot(_input.T, _gradOutput) # YOUR CODE HERE
        self.gradBias = np.sum(_gradOutput, axis=0) # YOUR CODE HERE
        
        self._gradInput = np.dot(_gradOutput, self.weight.T) # YOUR CODE HERE
        return self._gradInput
        
    def parameters(self):
        """
        Return weight and bias and their g
        """
        return [self.weight, self.bias], [self.gradWeight, self.gradBias]

In [184]:
class ReLU(Module):
    """
    ReLU activation, not trainable.
    """
    def __init__(self):
        Module.__init__(self)
        return
    
    def forward(self, _input):
        """
        output = max(0, input)
        
        _input:
        N x d matrix
        """
        self._output = np.maximum(0, _input) #YOUR CODE HERE
        return self._output
    
    def backward(self, _input, _gradOutput):
        """
        gradInput = gradOutput * mask
        mask = _input > 0
        
        _input:
        N x d matrix
        
        _gradOutput:
        N x d matrix
        """
        self._gradInput = _gradOutput * (_input > 0).astype(float) # YOUR CODE HERE
        return self._gradInput
        
    def parameters(self):
        """
        No trainable parametersm, return None
        """
        return None, None

In [185]:
# Optional
class Logistic(Module):
    """
    Logistic activation, not trainable.
    """
    def __init__(self):
        Module.__init__(self)
        return
    
    def forward(self, _input):
        self._output = 0 #YOUR CODE HERE
        return self._output
    
    def backward(self, _input, _gradOutput):
        self._gradInput = 0 #YOUR CODE HERE
        return self._gradInput
        
    def parameters(self):
        return None, None

In [186]:
# Optional
class Tanh(Module):
    """
    Tanh activation, not trainable.
    """
    def __init__(self):
        Module.__init__(self)
        return
    
    def forward(self, _input):
        self._output = 0 #YOUR CODE HERE
        return self._output
    
    def backward(self, _input, _gradOutput):
        self._gradInput = 0 #YOUR CODE HERE
        return self._gradInput
        
    def parameters(self):
        return None, None

In [187]:
# Optional
class Dropout(Module):
    """
    A dropout layer
    """
    def __init__(self, p = 0.5):
        Module.__init__(self)
        self.p = p #self.p is the drop rate, if self.p is 0, then it's a identity layer
        
    def forward(self, _input):
        self._output = _input
        # YOUR CODE HERE
        # Need to take care of training mode and evaluation mode
        return self._output
    
    def backward(self, _input, _gradOutput):
        self._gradInput = _gradOutput
        #YOUR CODE HERE
        return self._gradInput
    
    def parameters(self):
        """
        No trainable parameters.
        """
        return None, None

In [188]:
class SoftMaxLoss(object):
    def __init__(self):
        return
        
    def forward(self, _input, _label):
        """
        Softmax and cross entropy loss layer. Should return a scalar, since it's a
        loss. (It's almost identical to what in hw2)

        _input: N x C
        _labels: N x C, one-hot

        Returns: loss (scalar)
        """
        #YOUR CODE HERE

        exp_input = np.exp(_input - np.max(_input, axis=1, keepdims=True))
        softmax = exp_input / np.sum(exp_input, axis=1, keepdims=True)
        epsilon = 1e-9
        self._output = -np.sum(_label * np.log(softmax + epsilon)) / len(_input)

        return self._output
    
    def backward(self, _input, _label):
        
        exp_input = np.exp(_input - np.max(_input, axis=1, keepdims=True))
        softmax = exp_input / np.sum(exp_input, axis=1, keepdims=True)
        
        self._gradInput = (softmax - _label) / len(_input) #YOUR CODE HERE
        return self._gradInput

In [189]:
# Test softmaxloss, the relative error should be small enough
def test_sm():
    crit = SoftMaxLoss()
    gt = np.zeros((3, 10))
    gt[np.arange(3), np.array([1,2,3])] = 1
    x = np.random.random((3,10))
    def test_f(x):
        return crit.forward(x, gt)

    crit.forward(x, gt)

    gradInput = crit.backward(x, gt)
    gradInput_num = numeric_gradient(test_f, x, 1, 1e-6)
    #print(gradInput)
    #print(gradInput_num)
    print(relative_error(gradInput, gradInput_num, 1e-8))
    
test_sm()

9.05680276190978e-09


In [190]:
# Test modules, all the relative errors should be small enough
def test_module(model):

    model.evaluate()

    crit = TestCriterion()
    gt = np.random.random((3,10))
    x = np.random.random((3,10))
    def test_f(x):
        return crit.forward(model.forward(x), gt)

    gradInput = model.backward(x, crit.backward(model.forward(x), gt))
    gradInput_num = numeric_gradient(test_f, x, 1, 1e-6)
    print(relative_error(gradInput, gradInput_num, 1e-8))

# Test fully connected
model = FullyConnected(10, 10)
test_module(model)

# Test ReLU
model = ReLU()
test_module(model)

# Test Dropout
# model = Dropout()
# test_module(model)
# You can only test dropout in evaluation mode.

# Test Sequential
model = Sequential()
model.add(FullyConnected(10, 10))
model.add(ReLU())
#model.add(Dropout())
test_module(model)

1.7523407956079448e-08
5.962448038856628e-10
1.4172931374988576e-08


In [191]:
# Test gradient descent, the loss should be lower and lower
trainX = np.random.random((10,5))

model = Sequential()
model.add(FullyConnected(5, 3))
model.add(ReLU())
#model.add(Dropout())
model.add(FullyConnected(3, 1))

crit = TestCriterion()

it = 0
state = None
while True:
    output = model.forward(trainX)
    loss = crit.forward(output, None)
    if it % 100 == 0:
        print(loss)
    doutput = crit.backward(output, None)
    model.backward(trainX, doutput)
    params, gradParams = model.parameters()
    sgdmom(params, gradParams, 0.01, 0.8)
    if it > 1000:
        break
    it += 1

0.2095338882601591
0.019452571838974732
0.01747750382665477
0.010995051318409246
0.01455373056462134
0.007505352700255541
0.010238367867662784
0.012505628683159381
0.014465886106835423
0.011445656341263368
0.012812177436855915


Now we start to work on real data.

In [192]:
import MNIST_utils
data_fn = "CLEAN_MNIST_SUBSETS.h5"

# We only consider large set this time
print("Load large trainset.")
Xlarge,Ylarge = MNIST_utils.load_data(data_fn, "large_train")
print(Xlarge.shape)
print(Ylarge.shape)

print("Load valset.")
Xval,Yval = MNIST_utils.load_data(data_fn, "val")
print(Xval.shape)
print(Yval.shape)

Load large trainset.
(7000, 576)
(7000, 10)
Load valset.
(2000, 576)
(2000, 10)


In [193]:
def predict(X, model):
    """
    Evaluate the soft predictions of the model.
    Input:
    X : N x d array (no unit terms)
    model : a multi-layer perceptron
    Output:
    yhat : N x C array
        yhat[n][:] contains the score over C classes for X[n][:]
    """
    return model.forward(X)

def error_rate(X, Y, model):
    """
    Compute error rate (between 0 and 1) for the model
    """
    model.evaluate()
    res = 1 - (model.forward(X).argmax(-1) == Y.argmax(-1)).mean()
    model.training()
    return res

from copy import deepcopy

def runTrainVal(X,Y,model,Xval,Yval,trainopt):
    """
    Run the train + evaluation on a given train/val partition
    trainopt: various (hyper)parameters of the training procedure
    During training, choose the model with the lowest validation error. (early stopping)
    """
    
    eta = trainopt['eta']
    
    N = X.shape[0] # number of data points in X
    
    # Save the model with lowest validation error
    minValError = np.inf
    saved_model = None
    
    shuffled_idx = np.random.permutation(N)
    start_idx = 0
    for iteration in range(trainopt['maxiter']):
        if iteration % int(trainopt['eta_frac'] * trainopt['maxiter']) == 0:
            eta *= trainopt['etadrop']
        # form the next mini-batch
        stop_idx = min(start_idx + trainopt['batch_size'], N)
        batch_idx = range(N)[int(start_idx):int(stop_idx)]
        bX = X[shuffled_idx[batch_idx],:]
        bY = Y[shuffled_idx[batch_idx],:]

        score = model.forward(bX)
        loss = crit.forward(score, bY)
        # print(loss)
        dscore = crit.backward(score, bY)
        model.backward(bX, dscore)
        
        # Update the data using 
        params, gradParams = model.parameters()
        sgdmom(params, gradParams, eta, weight_decay = trainopt['lambda'])    
        start_idx = stop_idx % N
        
        if (iteration % trainopt['display_iter']) == 0:
            #compute train and val error; multiply by 100 for readability (make it percentage points)
            trainError = 100 * error_rate(X, Y, model)
            valError = 100 * error_rate(Xval, Yval, model)
            print('{:8} batch loss: {:.3f} train error: {:.3f} val error: {:.3f}'.format(iteration, loss, trainError, valError))
            
            if valError < minValError:
                saved_model = deepcopy(model)
                minValError = valError
        
    return saved_model, minValError, trainError

In [197]:
def build_model(input_size, hidden_size, output_size, activation_func = 'ReLU', dropout = 0):
    """
    Build the model:
    input_size: the dimension of input data
    hidden_size: the dimension of hidden vector
    output_size: the output size of final layer.
    activation_func: ReLU, Logistic, Tanh, etc. (Need to be implemented by yourself)
    dropout: the dropout rate: if dropout == 0, this is equivalent to no dropout
    """
    model = Sequential() #YOUR CODE HERE

    if hidden_size != 0:
        model.add(FullyConnected(input_size, hidden_size))
        if activation_func == 'ReLU':
            model.add(ReLU())
        if dropout != 0:
            model.add(Dropout(dropout))
    model.add(FullyConnected(hidden_size if hidden_size != 0 else input_size, output_size))
    
    return model

In [195]:
# -- training options
trainopt = {
    'eta': .1,   # initial learning rate
    'maxiter': 20000,   # max number of iterations (updates) of SGD
    'display_iter': 500,  # display batch loss every display_iter updates
    'batch_size': 100,  
    'etadrop': .5, # when dropping eta, multiply it by this number (e.g., .5 means halve it)
    'eta_frac': .25  #
}

NFEATURES = Xlarge.shape[1]

# we will maintain a record of models trained for different values of lambda
# these will be indexed directly by lambda value itself
trained_models = dict()

# set the (initial?) set of lambda values to explore
lambdas = np.array([0, 0.001, 0.01, 0.1])
hidden_sizes = np.array([10])
    
for lambda_ in lambdas:
    for hidden_size_ in hidden_sizes:
        trainopt['lambda'] = lambda_
        model = build_model(NFEATURES, hidden_size_, 10, dropout = 0)
        crit = SoftMaxLoss()
        # -- model trained on large train set
        trained_model,valErr,trainErr = runTrainVal(Xlarge, Ylarge, model, Xval, Yval, trainopt)
        trained_models[(lambda_, hidden_size_)] = {'model': trained_model, "val_err": valErr, "train_err": trainErr }
        print('train set model: -> lambda= %.4f, train error: %.2f, val error: %.2f' % (lambda_, trainErr, valErr))
    
best_trained_lambda = 0.
best_trained_model = None
best_trained_val_err = 100.
for (lambda_, hidden_size_), results in trained_models.items():
    print('lambda= %.4f, hidden size: %5d, val error: %.2f' %(lambda_, hidden_size_, results['val_err']))
    if results['val_err'] < best_trained_val_err:
        best_trained_val_err = results['val_err']
        best_trained_model = results['model']
        best_trained_lambda = lambda_

print("Best train model val err:", best_trained_val_err)
print("Best train model lambda:", best_trained_lambda)

       0 batch loss: 2.296 train error: 89.500 val error: 90.100


     500 batch loss: 0.431 train error: 11.229 val error: 10.300
    1000 batch loss: 0.342 train error: 8.843 val error: 8.750
    1500 batch loss: 0.302 train error: 7.429 val error: 7.950
    2000 batch loss: 0.340 train error: 7.029 val error: 7.950
    2500 batch loss: 0.349 train error: 6.557 val error: 7.700
    3000 batch loss: 0.251 train error: 6.057 val error: 7.650
    3500 batch loss: 0.155 train error: 5.686 val error: 7.200
    4000 batch loss: 0.150 train error: 5.343 val error: 7.400
    4500 batch loss: 0.187 train error: 5.029 val error: 7.400
    5000 batch loss: 0.226 train error: 4.743 val error: 7.750
    5500 batch loss: 0.236 train error: 4.500 val error: 7.500
    6000 batch loss: 0.179 train error: 4.229 val error: 7.250
    6500 batch loss: 0.176 train error: 4.329 val error: 7.350
    7000 batch loss: 0.112 train error: 4.014 val error: 7.100
    7500 batch loss: 0.098 train error: 4.029 val error: 7.450
    8000 batch loss: 0.136 train error: 3.743 val err

In [196]:
#Generate a Kaggle submission file using `model`
kaggleX = MNIST_utils.load_data(data_fn, 'kaggle')
kaggleYhat = predict(kaggleX, best_trained_model).argmax(-1)
save_submission('submission-mnist.csv', kaggleYhat)

Saved: submission-mnist.csv


I used ReLU as the activation function as this is the more common activation function used for the MNIST problem. I have included the validation errors as well as the best train model validation error and lambda value for the three different update algorithms. On Kaggle, the model that used sgdmom performed the best.

sgd:

lambda= 0.0000, hidden size:    10, val error: 6.90
lambda= 0.0010, hidden size:    10, val error: 6.60
lambda= 0.0100, hidden size:    10, val error: 7.25
lambda= 0.1000, hidden size:    10, val error: 7.60
Best train model val err: 6.599999999999994
Best train model lambda: 0.001

sgdm:

lambda= 0.0000, hidden size:    10, val error: 7.75
lambda= 0.0010, hidden size:    10, val error: 7.25
lambda= 0.0100, hidden size:    10, val error: 7.40
lambda= 0.1000, hidden size:    10, val error: 7.00
Best train model val err: 6.999999999999995
Best train model lambda: 0.1

sgdmom:

lambda= 0.0000, hidden size:    10, val error: 7.10
lambda= 0.0010, hidden size:    10, val error: 7.65
lambda= 0.0100, hidden size:    10, val error: 6.80
lambda= 0.1000, hidden size:    10, val error: 6.85
Best train model val err: 6.799999999999995
Best train model lambda: 0.01